# <font color="#747a7f">7</font> Visualização Cartográfica

*“A elaboração de mapas é uma das atividades intelectuais mais antigas da humanidade e também uma das mais complexas, com teoria científica, representação gráfica, fatos geográficos e considerações práticas combinados de maneiras variadas e incessantes.”* &mdash; [H. J. Steward](https://books.google.com/books?id=cVy1Ms43fFYC)

A Cartografia &ndash; o estudo e a prática de fazer mapas &ndash; tem uma rica história que abrange séculos de descobertas e design. A visualização cartográfica utiliza técnicas de mapeamento para transmitir dados contendo informações espaciais, como locais, rotas ou trajetórias na superfície da Terra.

<img width="300px" align="right" src="https://gist.githubusercontent.com/jheer/c90d582ef5322582cf4960ec7689f6f6/raw/8dc92382a837ccc34c076f4ce7dd864e7893324a/latlon.png"/>

Aproximando a Terra como uma esfera, podemos denotar posições usando um sistema de coordenadas esféricas de *latitude* (ângulo em graus ao norte ou ao sul do *equador*) e *longitude* (ângulo em graus especificando a posição leste-oeste). Neste sistema, um *paralelo* é um círculo de latitude constante e um *meridiano* é um círculo de longitude constante. O [*Meridiano de Greenwich*](https://en.wikipedia.org/wiki/Prime_meridian) está a 0° de longitude e, por convenção, é definido para passar pelo Observatório Real em Greenwich, Inglaterra.

Para "achatar" uma esfera tridimensional em um plano bidimensional, devemos aplicar uma [projeção](https://en.wikipedia.org/wiki/Map_projection) que mapeia pares (`longitude`, `latitude`) para coordenadas (`x`, `y`). Semelhante a [escalas](https://github.com/uwdata/visualization-curriculum/blob/master/altair_scales_axes_legends.ipynb), as projeções mapeiam de um domínio de dados (posição espacial) para um intervalo visual (posição do pixel). No entanto, os mapeamentos de escala que vimos até agora aceitam um domínio unidimensional, enquanto as projeções de mapa são inerentemente bidimensionais.

Neste notebook, vamos apresentar o básico sobre criação de mapas e visualizar dados espaciais com Altair, incluindo:

- Formatos de dados para representar características geográficas,
- Técnicas de geo-visualização, como mapas de pontos, símbolos e coropléticos, e
- Uma revisão das projeções cartográficas comuns.

*Este notebook é parte do [data visualization curriculum](https://github.com/uwdata/visualization-curriculum).*


In [ ]:
import pandas as pd
import altair as alt
from vega_datasets import data

## <font color="#747a7f">7.1</font> Dados Geográficos: GeoJSON e TopoJSON


Até este ponto, nós trabalhamos com datasets formatados em JSON e CSV que correspondem a tabelas de dados compostas por linhas (registros) e colunas (campos). Para representar regiões geográficas (países, estados, *etc.*) e trajetórias (rotas de voo, linhas de metrô, *etc.*), nós precisamos expandir nosso repertório com formatos adicionais projetados para suportar geometrias ricas.

[GeoJSON](https://en.wikipedia.org/wiki/GeoJSON) modela características geográficas dentro de um formato JSON especializado. Um `feature` do GeoJSON pode incluir dados geométricos &ndash; como coordenadas `longitude`, `latitude` que compõem a fronteira de um país &ndash; além de atributos de dados adicionais.

Aqui está um objeto `feature` do GeoJSON para a fronteira do estado do Colorado, nos EUA:


~~~ json
{
  "type": "Feature",
  "id": 8,
  "properties": {"name": "Colorado"},
  "geometry": {
    "type": "Polygon",
    "coordinates": [
      [[-106.32056285448942,40.998675790862656],[-106.19134826714341,40.99813863734313],[-105.27607827344248,40.99813863734313],[-104.9422739227986,40.99813863734313],[-104.05212898774828,41.00136155846029],[-103.57475287338661,41.00189871197981],[-103.38093099236758,41.00189871197981],[-102.65589358559272,41.00189871197981],[-102.62000064466328,41.00189871197981],[-102.052892177978,41.00189871197981],[-102.052892177978,40.74889940428302],[-102.052892177978,40.69733266640851],[-102.052892177978,40.44003613055551],[-102.052892177978,40.3492571857556],[-102.052892177978,40.00333031918079],[-102.04930288388505,39.57414465707943],[-102.04930288388505,39.56823596836465],[-102.0457135897921,39.1331416175485],[-102.0457135897921,39.0466599009048],[-102.0457135897921,38.69751011321283],[-102.0457135897921,38.61478847120581],[-102.0457135897921,38.268861604631],[-102.0457135897921,38.262415762396685],[-102.04212429569915,37.738153927339205],[-102.04212429569915,37.64415206142214],[-102.04212429569915,37.38900413964724],[-102.04212429569915,36.99365914927603],[-103.00046581851544,37.00010499151034],[-103.08660887674611,37.00010499151034],[-104.00905745863294,36.99580776335414],[-105.15404227428235,36.995270609834606],[-105.2222388620483,36.995270609834606],[-105.7175614468747,36.99580776335414],[-106.00829426840322,36.995270609834606],[-106.47490250048605,36.99365914927603],[-107.4224761410235,37.00010499151034],[-107.48349414060355,37.00010499151034],[-108.38081766383978,36.99903068447129],[-109.04483707103458,36.99903068447129],[-109.04483707103458,37.484617466122884],[-109.04124777694163,37.88049961001363],[-109.04124777694163,38.15283644441336],[-109.05919424740635,38.49983761802722],[-109.05201565922046,39.36680339854235],[-109.05201565922046,39.49786885730673],[-109.05201565922046,39.66062637372313],[-109.05201565922046,40.22248895514744],[-109.05201565922046,40.653823231326896],[-109.05201565922046,41.000287251421234],[-107.91779872584989,41.00189871197981],[-107.3183866123281,41.00297301901887],[-106.85895696843116,41.00189871197981],[-106.32056285448942,40.998675790862656]]
    ]
  }
}
~~~

A `feature` inclui um objeto `properties`, que pode conter qualquer número de campos, além de um objeto `geometry`, que neste caso contém um único polígono composto por coordenadas `[longitude, latitude]` para a fronteira do estado. As coordenadas continuam para a direita por um tempo, caso queira rolar...

Para saber mais sobre os detalhes técnicos do GeoJSON, consulte a [especificação oficial do GeoJSON](http://geojson.org/) ou leia o [guia útil de Tom MacWright](https://macwright.org/2015/03/23/geojson-second-bite).


Uma desvantagem do GeoJSON como formato de armazenamento é que ele pode ser redundante, resultando em tamanhos de arquivos maiores. Considere: o Colorado faz fronteira com outros seis estados (sete se incluir o canto que toca o Arizona). Ao invés de usar listas de coordenadas separadas, sobrepondo-se para cada um desses estados, uma abordagem mais compacta seria codificar as fronteiras compartilhadas apenas uma vez, representando a *topologia* das regiões geográficas. Felizmente, é exatamente isso que o formato [TopoJSON](https://github.com/topojson/topojson/blob/master/README.md) faz!


Vamos carregar um arquivo TopoJSON dos países do mundo (com resolução de 110 metros):

In [ ]:
world = data.world_110m.url
world

'https://cdn.jsdelivr.net/npm/vega-datasets@v1.29.0/data/world-110m.json'

In [ ]:
world_topo = data.world_110m()

In [ ]:
world_topo.keys()

dict_keys(['type', 'transform', 'objects', 'arcs'])

In [ ]:
world_topo['type']

'Topology'

In [ ]:
world_topo['objects'].keys()

dict_keys(['land', 'countries'])

*Inspecione o objeto dicionário TopoJSON `world_topo` acima para ver seu conteúdo.*

Nos dados acima, a propriedade `objects` indica os elementos nomeados que nós podemos extrair dos dados: geometrias para todos os `countries` (países) ou um único polígono representando toda a `land` (terra) na Terra. Qualquer um destes pode ser desempacotado para dados GeoJSON que nós podemos então visualizar.

Como o TopoJSON é um formato especializado, precisamos instruir o Altair a analisar o formato TopoJSON, indicando qual objeto feature nomeado nós desejamos extrair da topologia. O seguinte código indica que nós queremos extrair as features GeoJSON do dataset `world` para o objeto `countries`:

~~~ js
alt.topo_feature(world, 'countries')
~~~

Este método `alt.topo_feature` se expande para o seguinte JSON do Vega-Lite:

~~~ json
{
  "values": world,
  "format": {"type": "topojson", "feature": "countries"}
}
~~~

Agora que podemos carregar dados geográficos, nós estamos prontos para começar a fazer mapas!
